## Preparación de datasets

En este *notebook* vamos a limpiar los datasets en bruto que se habían obtenido. El objetivo es obtener la versión final del dataset de ratings que usaremos para obtener los datasets de entrenamiento, test y validación.

In [ ]:
import os
import pandas as pd

# Ruta de datasets en bruto
raw_path = os.path.join(os.getcwd(), "../..", "datasets", "raw")

# Lectura del dataset de ratings
col_types = {
    'user_id': 'int32',
    'book_id': 'int32',
    'rating': 'int8',
}
ratings_df = pd.read_csv(raw_path + "/ratings_raw.csv", index_col=0, dtype=col_types)
print("# de ratings:", ratings_df.shape[0])

En el dataset de ratings original, las puntuaciones están en una escala del 1 al 5. Para que las puntuaciones caigan en el intervalo $(0, 1)$ debemos reescalar la columna `rating` del dataset de valoraciones de usuario. Seguimos la siguiente transformación lineal: $$y = \frac{x - 1}{5 - 1} = \frac{x - 1}{4},$$ donde $x$ es la rating original e $y$ su nuevo valor. 

Queremos que estén en esta escala por las métricas de similitud que posteriormente usaremos en el modelo para evaluarlo.

In [ ]:
# Ponemos el rating en una escala de 0 a 1
ratings_df['rating'] = ratings_df['rating'].apply(lambda x: (x - 1)/4.0)
ratings_df['rating'] = ratings_df['rating'].astype('float32')
ratings_df.head()

Vamos a cribar el dataset de ratings quedándonos con aquellos usuarios que tengan 20 o más libros que hayan valorado positivamente. Definimos como bien valorados aquellos libros con un rating mayor o igual a 0.75 (originalmente, el usuario le dio una puntuación >= 4). 

Todos los libros tienen un gran número de valoraciones, por lo que no tenemos que preocuparnos de seleccionar aquellos con un mínimo de ratings. Esto ya se analizó en el *notebook* sobre el dataset `goodbooks-10k`.

In [ ]:
# Seleccionamos los usuarios con k o más valoraciones positivas (rating >= 0.75)
k = 20
positive_ratings_df = ratings_df[ratings_df['rating'] >= 0.75]
positive_ratings_df = positive_ratings_df.groupby('user_id').filter(lambda x: len(x) >= k)
users_with_enough_positive_ratings = positive_ratings_df['user_id'].unique()
print(f"# de usuarios con al menos {k} ratings positivos:", users_with_enough_positive_ratings.shape[0])

In [ ]:
# Quitamos de ratings aquellos usuarios que no tienen suficientes valoraciones positivas
ratings_filtered_df = ratings_df[ratings_df['user_id'].isin(users_with_enough_positive_ratings)].copy()
ratings_filtered_df.sort_values(by=['user_id', 'book_id'], inplace=True)
ratings_filtered_df.reset_index(drop=True, inplace=True)
print("# de ratings:", ratings_filtered_df.shape[0])

In [ ]:
# Ruta de datasets preparados
ready_path = os.path.join(os.getcwd(), "../..", "datasets", "ready")

# Se guarda el dataset de ratings filtrado
ratings_filtered_df.to_csv(ready_path + "/ratings.csv", index=False)